# Clojupyter Demo

This notebook demonstrates some of the more advanced features of Clojupyter.

## Displaying HTML

To display HTML, you'll need to require a clojupyter helper function to change the cell output

In [1]:
(require '[clojupyter.misc.display :as display])
;; print some text
(println (str "Date:\t\t\t"(java.util.Date.)))
(println (apply format "Clojure version:\tv%d.%d" ((juxt :major :minor) *clojure-version*)));

Date:			Tue Feb 26 14:14:45 CET 2019
Clojure version:	v1.10


In [2]:
;; displaying html
(display/hiccup-html 
    [:ul 
     [:li "a " [:i "emphatic"] " idea"]
     [:li "a " [:b "bold"] " idea"]
     [:li "an " [:span {:style "text-decoration: underline;"} "important"] " idea"]])

a emphatic idea a bold idea an important idea

In [3]:
(display/hiccup-html
    [:svg {:height 100 :width 100 :xmlns "http://www.w3.org/2000/svg"}
            [:circle {:cx 50 :cy 40 :r 40 :fill "red"}]])

## Adding External Clojure Dependencies

You can fetch external Clojure dependcies using the clojupyter.misc.helper namespace.

In [4]:
(require '[clojupyter.misc.helper :as helper])
(helper/add-dependencies '[org.clojure/data.json "0.2.6"])
(require '[clojure.data.json :as json]);

In [5]:
(json/write-str {:a 1 :b [2, 3] :c "c"})

"{\"a\":1,\"b\":[2,3],\"c\":\"c\"}"

## Adding External Javascript Dependency

Since you can render arbitrary HTML using display/hiccup-html, it's pretty easy to use external Javascript libraries to do things like generate charts. Here's an example using Highcharts.

First, we use a cell to add javascript to the running notebook:

In [6]:
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(require '[oz.notebook.clojupyter :as oz]);

Now we define functions which takes Clojure data and converts it for use with the high-level Vega Lite graph library:

In [7]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn graph  [vs]    {:data {:values vs}, 
                      :mark :line
                      :encoding {:x {:field :Index}
                                 :y {:field :Value}}})
(def graph-it (comp graph (partial map datapt) (partial apply map vector)));

Defining a generator for (slightly) random data...

In [8]:
(defn raw-data [] (map #(+ % (rand-int 25)) (range)))

#'user/raw-data

...we can create a graph using Vega Lite:

In [9]:
(->> [(range) (take 150 (raw-data))]
     graph-it
     oz/view!)